In [ ]:
Oimport numpy as np
import plotly.express as px
import plotly.graph_objects as go

**Поиск собственных функций и значений**

In [ ]:
h = 0.01

l = 5
ρ = 1
m = ρ * l / 4

x = np.arange(0.0, l + h, h)
n = len(x)


# из вида граничных условий определяем константы

α_l = 1
α_r = 0
β_l = 0
β_r = 1

# задаем три диагонали матрицы оператора


def create_diagonals_L(n, h, α_l, β_l, α_r, β_r):

    main_diag = np.zeros(n)
    upper_diag = np.zeros(n - 1)
    lower_diag = np.zeros(n - 1)

    main_diag[0] = 1 + (2 * α_l * h) / (2 * β_l + α_l * h)
    main_diag[1:n-1] = 2
    main_diag[-1] = ρ * h / (ρ * h + m)

    upper_diag[0:n-1] = -1

    lower_diag[0:n-2] = -1
    lower_diag[-1] = - ρ * h / (ρ * h + m)

    return main_diag / h ** 2, upper_diag / h ** 2, lower_diag / h ** 2


main_diag, upper_diag, lower_diag = create_diagonals_L(n, h, α_l, β_l, α_r, β_r)

In [ ]:
# реализация метода прогонки

def A_k(x, A_k_, main_diag, upper_diag, lower_diag):
    for i in range(1, len(x) - 1):
        A_k_.append(- upper_diag[i] / (main_diag[i] + lower_diag[i - 1] * A_k_[i - 1]))
    A_k_.append(0)


def B_k(x, B_k_, A_k_, main_diag, upper_diag, lower_diag, X):
    for i in range(1, len(x) - 1):
        B_k_.append((X[i] - lower_diag[i - 1] * B_k_[i - 1]) / (main_diag[i] + lower_diag[i - 1] * A_k_[i - 1]))
    B_k_.append((X[-1] - lower_diag[-1] * B_k_[len(x) - 2]) / (main_diag[-1] + lower_diag[-1] * A_k_[len(x) - 2]))


def run_through_method(main_diag, upper_diag, lower_diag, X, x):
    c_0 = upper_diag[0]
    b_0 = main_diag[0]
    f_0 = X[0]

    A_k_ = [-c_0 / b_0]
    A_k(x, A_k_, main_diag, upper_diag, lower_diag)

    B_k_ = [f_0 / b_0]
    B_k(x, B_k_, A_k_, main_diag, upper_diag, lower_diag, X)

    X_new = np.zeros(len(x))
    X_new[-1] = B_k_[len(x) - 1]
    for i in range(len(x) - 2, -1, -1):
        X_new[i] = B_k_[i] + A_k_[i] * X_new[i + 1]

    return X_new

In [ ]:
# реализация умножения на трехдиагональную матрицу

def apply_three_diagonal(main_diag, upper_diag, lower_diag, X):
    result = np.zeros(len(main_diag))

    result[0] = main_diag[0] * X[0] + upper_diag[0] * X[1]
    for i in range(1, len(main_diag) - 1):
        result[i] = lower_diag[i - 1] * X[i - 1] + main_diag[i] * X[i] + upper_diag[i] * X[i + 1]
    result[len(main_diag) - 1] = lower_diag[len(main_diag) - 2] * X[len(main_diag) - 2] + main_diag[len(main_diag) - 1] * X[len(main_diag) - 1]

    return result

In [ ]:
# реализация итерационного метода Рэлея

def rayleigh_method(main_diag, upper_diag, lower_diag, x, l, ρ, m, num_eigenfunctions=5, max_iter=1000, ε=1e-16):
    eigenvalues = []
    eigenfunctions = []
    λ = [0, 1, 2, 4, 7] # сами выбираем начальные приближения собственных значений

    for i in range(num_eigenfunctions):
        X = np.sin(np.sqrt(λ[i]) * x - ε)
        X = X / np.linalg.norm(X)

        for _ in range(max_iter):
            X_wave = run_through_method(main_diag - λ[i], upper_diag, lower_diag, X, x)
            X_wave = X_wave / np.linalg.norm(X_wave)

            λ_new = np.dot(apply_three_diagonal(main_diag, upper_diag, lower_diag, X_wave), X_wave) / np.dot(X_wave, X_wave)

            if abs(λ[i] - λ_new) < ε:
                break

            λ[i] = λ_new
            X = X_wave

        eigenvalues.append(λ[i])
        eigenfunctions.append(X)

    return eigenvalues, eigenfunctions


eigenvalues, eigenfunctions = rayleigh_method(main_diag, upper_diag, lower_diag, x, l, ρ, m)
print(eigenvalues)

[0.06376909022284337, 0.617762023630598, 1.8528210602310926, 3.8423058474985, 6.60870766621264]


**Собственные функции и значения в явном виде**

In [ ]:
# функция из уравнения на собственные значения
def F(x):
    return x * np.tan(x) - 4


# метод дихотомии
def the_dichotomy_method(x_beginning, x_ending, ε):
    a = x_beginning
    b = x_ending

    for i in range(round(np.log2((x_ending - x_beginning) / ε)) + 1):
        x = (a + b) / 2
        if F(x) * F(a) == 0:
            return x
        elif F(x) * F(a) < 0:
            b = x
        else:
            a = x

    return x


def find_roots(n, ε):
    roots = []
    a = 0

    for i in range(n):
        b = a + np.pi # тут пользуемся периодичностью тангенса
        roots.append(the_dichotomy_method(a, b, ε))
        a = b

    return roots


μ = find_roots(5, 1e-16)
print(μ)

[1.2645915712878018, 3.935161652940459, 6.814010343163549, 9.811878138915574, 12.867755694862307]


Держим в голове, что μ * tg(μ) - 4 - четная функция, поэтому μ = [±1.2645915712878018, ±3.935161652940459, ±6.814010343163549, ±9.811878138915574, ±12.867755694862307], однако собственные значения - это μ^2/l^2, поэтому неважно...

In [ ]:
λ_real = [(μ_i / l) ** 2 for μ_i in μ]
print(λ_real)

[0.06396767368688605, 0.6194198893909233, 1.857229478269593, 3.850918104517174, 6.623165464906452]


**Визуализация**

In [ ]:
fig = go.Figure()

for i, λ in enumerate(λ_real):
    fig.add_trace(go.Scatter(x=x, y=np.sin(np.sqrt(λ) * x) / np.linalg.norm(np.sin(np.sqrt(λ) * x)), name=f"λ={λ}", mode='lines'))

for i, eigenfunction in enumerate(eigenfunctions):
    fig.add_trace(go.Scatter(x=x, y=eigenfunction, name=f"λ={eigenvalues[i]}",
                             line=dict(color='black', dash='dash'),
                             mode='lines'))

fig.update_layout(
    title='Normalized eigenfunctions X(x) corresponding to the minimum eigenvalues λ',
    xaxis_title='x',
    yaxis_title='X(x)'
)

fig.show()

**Вычисление погрешностей и указание порядка аппроксимации**

In [ ]:
h = 0.1

ln_h = []
fault_array = []

for i in range(3):
    x = np.arange(0.0, l + h, h)
    n = len(x)

    main_diag, upper_diag, lower_diag = create_diagonals_L(n, h, α_l, β_l, α_r, β_r)

    eigenvalues, eigenfunctions = rayleigh_method(main_diag, upper_diag, lower_diag, x, l, ρ, m)

    errors = []
    for j in range(len(eigenvalues)):
        analytical_solution = np.sin(np.sqrt(eigenvalues[j]) * x) / np.linalg.norm(np.sin(np.sqrt(eigenvalues[j]) * x))
        error = np.max(np.abs(analytical_solution - eigenfunctions[j]))
        errors.append(error)

    fault_array.append(np.log(np.max(errors)))

    ln_h.append(np.log(h))

    h /= 5

In [ ]:
tg = abs(fault_array[0] - fault_array[2]) / abs(ln_h[0] - ln_h[2])
print(tg)


fig = go.Figure(data=[go.Scatter(x=ln_h, y=fault_array, mode='lines', line=dict(color='black'))])

fig.update_layout(title='Evaluation of the accuracy of the method',
                  xaxis_title='ln(h)',
                  yaxis_title='ln(max_err)')

fig.show()

2.3644600443414636
